<a href="https://colab.research.google.com/github/Sicily-F/cagedbirdID/blob/main/11_Building_a_binary_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 11. Building a binary model

Heavily occluded images are typically more difficult to classify than unobstructed. It remains to be seen if occlusion causes a significant drop in model performance. To investigate if this is the case for birds, we built a binary classifier to distinguish between 'caged' and 'uncaged' images in our dataset. 'Caged' photos have cage bars obscuring the bird in the foreground, and 'uncaged photos' do not have bars obstructing the view of the bird in the foreground. The model architecture selected for this binary model was VGG-16, to separate photos into caged and uncaged.  

We trained this model for 100 epochs, with a batch size of 16, the Adam optimiser, and patience of 10 epochs. Using the pandas package (McKinney, 2020), the predictions from our model were rounded to either 0 (caged) or 1 (uncaged), then exported as a .csv file, which was then used to sort the photos into caged and uncaged datasets. In total, 1,871 images were detected as uncaged of the original images in the TOT_SP_37 dataset (31.38%, 5,963 images).

In [ ]:
# This code was inspired from: https://www.kaggle.com/raulcsimpetru/vgg16-binary-classification

import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.models import Model

In [ ]:
TRAIN_DIR = 'F:/newforegroundanalysis/train' 
VAL_DIR = 'F:/newforegroundanalysis/val'

# A new test directory of all the photos, this was a folder of all the raw ground-truth data - then we sorted all these images
test_all = 'F:/allspecies_unaugmented_cropped_tobinary'


RANDOM_SEED = 1
IMG_SIZE = (224, 224) 
BATCH_SIZE = 16 


train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    brightness_range=[0.5, 1.25],
    horizontal_flip=True,
    vertical_flip=True,
    preprocessing_function=tf.keras.applications.vgg16.preprocess_input
)

test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=tf.keras.applications.vgg16.preprocess_input
)


train_gen = train_datagen.flow_from_directory(
    TRAIN_DIR,
    color_mode='rgb',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    seed=RANDOM_SEED
)

val_gen = test_datagen.flow_from_directory(
    VAL_DIR,
    color_mode='rgb',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    seed=RANDOM_SEED
)

test_gen = test_datagen.flow_from_directory(
    test_all, #TEST_DIR
    target_size=IMG_SIZE, 
    batch_size= BATCH_SIZE,
    class_mode='binary',
    shuffle=False
)

# This needs to be downloaded beforehand and placed in the working directory
vgg16_weight_path = 'F:/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5'
base_model = tf.keras.applications.VGG16(
    weights=vgg16_weight_path,
    include_top=False,
    input_shape=IMG_SIZE + (3,)
)

model = tf.keras.models.Sequential()
model.add(base_model)
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.layers[0].trainable = False

model.compile(
    loss='binary_crossentropy',
    optimizer=tf.keras.optimizers.Adam(lr=0.00001), # lr was only 0.1 before
    metrics=['accuracy']
)

model.summary()

EPOCHS = 100

filepath = "F:/VGGBINARYNEWDATASET.h5"

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
# stop training if there is no improvement in model for 15 consecutives epochs.
early_stopping_monitor = EarlyStopping(patience=15)
callbacks_list = [checkpoint, early_stopping_monitor]


history = model.fit(
    train_gen,
    steps_per_epoch=train_gen.samples // BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=val_gen,
    validation_steps=val_gen.samples // BATCH_SIZE,
    callbacks=[callbacks_list]
)

# Load the model to either check the test accuracy or keep training
model = load_model('F:/VGGBINARYNEWDATASET.h5')
from tensorflow.keras.optimizers import Adam

opt = Adam(lr=0.0001)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['acc'])			 
			 
filepath = "VGGBINARYNEWDATASET2ndtraining.h5" #

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
# stop training if there is no improvement in model for 6 consecutives epochs.
early_stopping_monitor = EarlyStopping(patience=6)
callbacks_list = [checkpoint, early_stopping_monitor]

EPOCHS = 30
model_history_2=model.fit(
	train_gen,
    steps_per_epoch = train_gen.samples // BATCH_SIZE,
    validation_data = val_gen, 
    validation_steps = val_gen.samples // BATCH_SIZE,
    epochs = EPOCHS,
    shuffle = True,
    callbacks=callbacks_list)


print(model.summary())


#print("Training Done")
#model.save("F:/binarymodel3.h5")
test_loss, test_acc = model.evaluate(test_gen)
print('Test accuracy:', test_acc)
 

#Test accuracy: 0. on all the data: Test accuracy: 0.5770551562309265


test_gen.reset() #maybe not used 
pred=model.predict(test_gen,verbose=1,steps=len(test_gen)) #306/BATCH_SIZE
predicted_class_indices=np.argmax(pred,axis=1)

labels = (train_gen.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]
filenames=test_gen.filenames

import pandas as pd                     
#also check here for options: https://stackoverflow.com/questions/56695299/how-should-i-use-mode-predict-generator-to-evaluate-model-performance-in-a-confu

cl = np.round(pred)

filenames=test_gen.filenames

results=pd.DataFrame({"file":filenames,"pr":pred[:,0], "class":cl[:,0]})

results.to_csv("F:/results11.csv",index=False)


In [ ]:
model=load_model("F:/VGGBINARYNEWDATASET.h5")

test_gen.reset() 
pred=model.predict(test_gen,verbose=1,steps=len(test_gen)) 
predicted_class_indices=np.argmax(pred,axis=1)

labels = (train_gen.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]
filenames=test_gen.filenames

cl = np.round(pred)

filenames=test_gen.filenames

results=pd.DataFrame({"file":filenames,"pr":pred[:,0], "class":cl[:,0]})

results.to_csv("results22042.csv",index=False)


Originally, the 'results22042.csv' from Tensorflow has a column for the % accuracy. We had to delete that column and rename the class to class_pred 
becaue the class is a function in Python, I then had to find and 
replace the allphotos\from before the filename (so it was the relative not absolute file path), then the sorting code below works, into files of what the classification is (caged or uncaged).

In [ ]:
SOURCE_ROOT = 'F:/still_convert/new'
DEST_ROOT = 'F:/extra_sorted'


with open('results22042.csv') as infile:
    next(infile)  # Skip the header row
    reader = csv.reader(infile)
    seen = set()
    for file, class_pred in reader:
        # Create a new directory if needed
        if class_pred not in seen:
            os.mkdir(os.path.join(DEST_ROOT, class_pred))
            seen.add(class_pred)
        src = os.path.join(SOURCE_ROOT, file)
        dest = os.path.join(DEST_ROOT, class_pred, file)
        try:
            os.rename(src, dest)
        except WindowsError as e:
            print (e)                
            

Once we had the photos sorted we could create our artifical images, which can be viewed in the next code file.